In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('main.csv')

In [3]:
df = df.drop(columns=['Collector ID', 'IP Address', 'Email Address', 'Custom Data 1','Respondent ID','First Name','Last Name'])
print(df.isnull().sum())

Start Date                                                        1
End Date                                                          1
I identify as having a mental illness                             0
Education                                                         0
I have my own computer separate from a smart phone                0
I have been hospitalized before for my mental illness             0
How many days were you hospitalized for your mental illness      37
I am currently employed at least part-time                        0
I am legally disabled                                             0
I have my regular access to the internet                          0
I live with my parents                                            0
I have a gap in my resume                                         0
Total length                                                      0
Annual income (including any social welfare programs) in USD      0
I am unemployed                                 

In [4]:
df.rename(columns={'Unnamed: 28': 'Anxiety'}, inplace=True)
df.rename(columns={'Unnamed: 28': 'Anxiety'}, inplace=True)
df.rename(columns={'Unnamed: 29': 'Depression'}, inplace=True)
df.rename(columns={'Unnamed: 31': 'Mood Swings'}, inplace=True)
df.rename(columns={'Unnamed: 32': 'Panic attacks'}, inplace=True)
df.rename(columns={'Unnamed: 33': 'Compulsive behavior'}, inplace=True)
df.rename(columns={'Unnamed: 30': 'Obsessive thinking'}, inplace=True)
df.rename(columns={'Unnamed: 34': 'Tiredness'}, inplace=True)

In [5]:
df['Anxiety'].fillna(0, inplace=True)
df['Anxiety'] = df['Anxiety'].apply(lambda x: 1 if x != 0 else 0)
df['Depression'].fillna(0, inplace=True)
df['Depression'] = df['Depression'].apply(lambda x: 1 if x != 0 else 0)
df['Mood Swings'].fillna(0, inplace=True)
df['Mood Swings'] = df['Mood Swings'].apply(lambda x: 1 if x != 0 else 0)
df['Panic attacks'].fillna(0, inplace=True)
df['Panic attacks'] = df['Panic attacks'].apply(lambda x: 1 if x != 0 else 0)
df['Compulsive behavior'].fillna(0, inplace=True)
df['Compulsive behavior'] = df['Compulsive behavior'].apply(lambda x: 1 if x != 0 else 0)
df['Obsessive thinking'].fillna(0, inplace=True)
df['Obsessive thinking'] = df['Obsessive thinking'].apply(lambda x: 1 if x != 0 else 0)
df['Tiredness'].fillna(0, inplace=True)
df['Tiredness'] = df['Tiredness'].apply(lambda x: 1 if x != 0 else 0)
df['Lack of concentration'].fillna(0, inplace=True)
df['Lack of concentration'] = df['Lack of concentration'].apply(lambda x: 1 if x != 0 else 0)

In [6]:
df = df.iloc[1:]

In [7]:
df.rename(columns={'I identify as having a mental illness':'MentallyIll'}, inplace =True)
df.rename(columns={'I am unemployed':'Unemployed'}, inplace =True)
df.rename(columns={'Annual income (including any social welfare programs) in USD':'Annual'}, inplace =True)

In [8]:
excluded_values = ['0', '35', '65', '20', '78', '44', '60', '13', '8', '1', '91', '14', '5', '21', '99', '2', '6', '100', '42', '3', '28', '15', '10', '27', '43']
df['How many days were you hospitalized for your mental illness'] = pd.to_numeric(df['How many days were you hospitalized for your mental illness'], errors='coerce')
filtered_values = df[~df['How many days were you hospitalized for your mental illness'].isin(excluded_values)]
mean_hospitalization_days = filtered_values['How many days were you hospitalized for your mental illness'].mean()
print("Среднее количество дней госпитализации (без учета исключенных значений):", mean_hospitalization_days)

Среднее количество дней госпитализации (без учета исключенных значений): 3.276094276094276


In [9]:
top_two_incomes = df[df['Household Income'] != 'Prefer not to answer']['Household Income'].value_counts().index[:3]

In [10]:
df = df.replace(to_replace=['No', 'Yes'], value=[0, 1])

In [11]:
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['End Date'] = pd.to_datetime(df['End Date'])
df['duration'] = df['End Date'] - df['Start Date']

In [12]:
df = df.drop(['Start Date', 'End Date'], axis=1)

In [13]:
df['duration'] = pd.to_timedelta(df['duration'])
df['duration_seconds'] = df['duration'].dt.total_seconds()

In [14]:
df = df.drop(columns=['duration'])

In [15]:
mask = df['MentallyIll'] == 0
df.loc[mask, 'How many days were you hospitalized for your mental illness'] = df.loc[mask, 'How many days were you hospitalized for your mental illness'].fillna(0)

In [16]:
df.isnull().sum()

MentallyIll                                                     0
Education                                                       0
I have my own computer separate from a smart phone              0
I have been hospitalized before for my mental illness           0
How many days were you hospitalized for your mental illness     5
I am currently employed at least part-time                      0
I am legally disabled                                           0
I have my regular access to the internet                        0
I live with my parents                                          0
I have a gap in my resume                                       0
Total length                                                    0
Annual                                                          0
Unemployed                                                      0
I read outside of work and school                               0
Annual income from social welfare programs                      0
I receive 

In [17]:
age_mapping = {
    '18-29':1,
    '30-44':2,
    '45-60':3,
    '> 60':4
    
}
df['Age'] = df['Age'].map(age_mapping)
min_value = df['Age'].min()
max_value = df['Age'].max()
df['Age'] = (df['Age'] - min_value) / (max_value - min_value)

In [18]:
education_mapping = {
    'High School or GED': 1,
    'Some Phd': 8,
    'Completed Undergraduate': 4,
    'Some Undergraduate': 3,
    'Some\xa0Masters': 5,
    'Completed Masters': 6,
    'Completed Phd': 9,
    'Some highschool': 2
}
df['Education'] = df['Education'].map(education_mapping)
df['Education'] = (df['Education'] - df['Education'].min()) / (df['Education'].max() - df['Education'].min())

In [19]:
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

In [20]:
filtered_df = df[(df['How many days were you hospitalized for your mental illness'].isnull()) | 
                 (df['Region'].isnull()) |
                 (df['Household Income'] == 'Prefer not to answer') |
                 (df['Device Type'] == 'Other')]
df.drop(filtered_df.index, inplace=True)
filtered_df.to_csv('null_data.csv', index=False)

In [21]:
df1=pd.read_csv("null_data.csv")

In [22]:
df1['Device Type'] = df1['Device Type'].replace('Other', 'Windows Desktop / Laptop')

In [23]:
region_counts = df['Region'].value_counts()
most_common_region = region_counts.idxmax()
df1['Region'].fillna(most_common_region, inplace=True)

In [24]:
most_common_region 

'South Atlantic'

In [25]:
df1["Region"].isnull().value_counts()

False    46
Name: Region, dtype: int64

In [26]:
df1['How many days were you hospitalized for your mental illness'].fillna(mean_hospitalization_days, inplace=True)

In [27]:
import random
probabilities = [0.6, 0.4,0.1]
for index, row in df1.iterrows():
    if row['Household Income'] == 'Prefer not to answer':
        replacement_value = random.choices(top_two_incomes, weights=probabilities)[0]
        df1.at[index, 'Household Income'] = replacement_value

In [28]:
df['Household Income'] = df['Household Income'].str.replace('$', '', regex=True).str.replace(',', '', regex=True)

df['Household Income'] = df['Household Income'].str.replace('+', '-', regex=True)
df[['Lower_Household_Income', 'Upper_Household_Income']] = df['Household Income'].str.split('-', expand=True)

df['Lower_Household_Income'] = pd.to_numeric(df['Lower_Household_Income'], errors='coerce')
df['Upper_Household_Income'] = pd.to_numeric(df['Upper_Household_Income'], errors='coerce')

df['Upper_Household_Income'].fillna(999999, inplace=True)
df.drop('Household Income', axis=1, inplace=True)


In [29]:
df1['Household Income'] = df1['Household Income'].str.replace('$', '', regex=True).str.replace(',', '', regex=True)

df1['Household Income'] = df1['Household Income'].str.replace('+', '-', regex=True)
df1[['Lower_Household_Income', 'Upper_Household_Income']] = df1['Household Income'].str.split('-', expand=True)

df1['Lower_Household_Income'] = pd.to_numeric(df1['Lower_Household_Income'], errors='coerce')
df1['Upper_Household_Income'] = pd.to_numeric(df1['Upper_Household_Income'], errors='coerce')

df1['Upper_Household_Income'].fillna(999999, inplace=True)
df1.drop('Household Income', axis=1, inplace=True)


In [30]:
device_mapping = {
    'Android Phone / Tablet': 1,
    'MacOS Desktop / Laptop': 3,
    'Windows Desktop / Laptop': 2,
    'iOS Phone / Tablet': 3  
}
df['Device Type'] = df['Device Type'].map(device_mapping)
df['Device Type'] = (df['Device Type'] - df['Device Type'].min()) / (df['Device Type'].max() - df['Device Type'].min())


In [31]:
device_mapping = {
    'Android Phone / Tablet': 1,
    'MacOS Desktop / Laptop': 3,
    'Windows Desktop / Laptop': 2,
    'iOS Phone / Tablet': 3  
}
df1['Device Type'] = df1['Device Type'].map(device_mapping)
df1['Device Type'] = (df1['Device Type'] - df1['Device Type'].min()) / (df1['Device Type'].max() - df1['Device Type'].min())


In [32]:
region_mapping = {
    'New England': 1,
    'Middle Atlantic': 2,
    'East North Central': 3,
    'West North Central': 4,
    'East South Central': 5,
    'South Atlantic': 6,
    'West South Central': 7,
    'Pacific': 8,
    'Mountain': 9
}
df['Region'] = df['Region'].map(region_mapping)
df['Region'] = (df['Region'] - df['Region'].min()) / (df['Region'].max() - df['Region'].min())


In [33]:
region_mapping = {
    'New England': 1,
    'Middle Atlantic': 2,
    'East North Central': 3,
    'West North Central': 4,
    'East South Central': 5,
    'South Atlantic': 6,
    'West South Central': 7,
    'Pacific': 8,
    'Mountain': 9
}
df1['Region'] = df1['Region'].map(region_mapping)
df1['Region'] = (df1['Region'] - df1['Region'].min()) / (df1['Region'].max() - df1['Region'].min())


In [34]:
df.isnull().sum()

MentallyIll                                                     0
Education                                                       0
I have my own computer separate from a smart phone              0
I have been hospitalized before for my mental illness           0
How many days were you hospitalized for your mental illness     0
I am currently employed at least part-time                      0
I am legally disabled                                           0
I have my regular access to the internet                        0
I live with my parents                                          0
I have a gap in my resume                                       0
Total length                                                    0
Annual                                                          0
Unemployed                                                      0
I read outside of work and school                               0
Annual income from social welfare programs                      0
I receive 

In [35]:
df

,MentallyIll,Education,I have my own computer separate from a smart phone,I have been hospitalized before for my mental illness,How many days were you hospitalized for your mental illness,I am currently employed at least part-time,I am legally disabled,I have my regular access to the internet,I live with my parents,I have a gap in my resume,...,Panic attacks,Compulsive behavior,Tiredness,Age,Gender,Region,Device Type,duration_seconds,Lower_Household_Income,Upper_Household_Income
1,0,0.000,0,0,0.0,0,0,1,0,1,...,1,0,0,0.333333,1,1.000,0.0,188.0,25000,49999.0
2,1,0.875,1,0,0.0,1,0,1,0,0,...,1,0,1,0.000000,1,0.500,1.0,65.0,50000,74999.0
3,0,0.375,1,0,0.0,1,0,1,0,0,...,0,0,0,0.333333,1,0.875,1.0,141.0,150000,174999.0
4,0,0.250,1,0,0.0,0,0,1,1,1,...,0,0,0,0.333333,1,0.000,0.5,77.0,25000,49999.0
5,1,0.375,1,1,35.0,1,1,1,0,1,...,1,1,1,0.333333,1,0.250,1.0,142.0,25000,49999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1,0.375,0,0,0.0,0,0,1,0,0,...,0,0,0,0.000000,0,1.000,1.0,80.0,50000,74999.0
331,0,0.250,1,0,0.0,1,0,1,1,0,...,0,0,0,0.000000,1,0.875,0.5,91.0,50000,74999.0
332,0,0.250,1,0,0.0,1,0,1,0,0,...,0,0,0,1.000000,0,0.375,0.5,341.0,10000,24999.0
333,1,0.250,0,1,1.0,0,1,1,1,1,...,1,1,1,0.000000,0,0.750,0.0,194.0,0,9999.0


In [36]:
df1

,MentallyIll,Education,I have my own computer separate from a smart phone,I have been hospitalized before for my mental illness,How many days were you hospitalized for your mental illness,I am currently employed at least part-time,I am legally disabled,I have my regular access to the internet,I live with my parents,I have a gap in my resume,...,Panic attacks,Compulsive behavior,Tiredness,Age,Gender,Region,Device Type,duration_seconds,Lower_Household_Income,Upper_Household_Income
0,1,0.375,1,0,0.000000,1,0,1,0,0,...,0,1,0,0.000000,1,0.875,1.0,63.0,25000,49999
1,0,0.250,1,0,0.000000,0,0,1,1,0,...,1,0,0,0.000000,1,0.250,0.5,83.0,25000,49999
2,1,0.250,1,0,3.276094,1,0,1,0,1,...,1,1,1,0.333333,1,0.250,1.0,182.0,25000,49999
3,0,0.125,1,0,0.000000,0,0,1,1,0,...,0,0,0,0.333333,1,0.875,0.5,142.0,50000,74999
4,0,0.000,1,0,0.000000,0,0,1,1,0,...,0,0,1,0.000000,1,0.875,0.0,141.0,50000,74999
5,0,0.625,1,0,0.000000,0,0,0,0,0,...,0,0,0,0.000000,1,0.125,0.5,56.0,25000,49999
6,1,0.000,1,0,0.000000,1,0,1,0,1,...,1,0,0,0.333333,1,0.500,0.5,75.0,50000,74999
7,0,0.375,1,0,0.000000,0,0,1,1,0,...,0,0,1,0.333333,1,0.625,0.5,287.0,10000,24999
8,0,0.000,1,0,0.000000,1,0,1,0,0,...,0,0,0,0.333333,1,0.875,1.0,55.0,25000,49999
9,0,1.000,1,0,0.000000,0,0,1,0,0,...,0,0,0,1.000000,1,0.625,0.5,152.0,75000,99999


In [37]:
from sklearn.model_selection import train_test_split
stratify_column = 'Unemployed'
train_df, test_df = train_test_split(df, test_size=0.35, stratify=df[stratify_column], random_state=42)
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

In [38]:
train_df=pd.read_csv("train_data.csv")

In [39]:
merged_df = pd.concat([df1, train_df], ignore_index=True)

In [40]:
merged_df.fillna(0, inplace=True)

In [41]:
merged_df.to_csv('trainschybou.csv', index=False)

In [42]:
merged_df['Unemployed'].value_counts()

0    171
1     62
Name: Unemployed, dtype: int64